In [ ]:
import pandas as pd
import numpy as np
import optuna
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sklearn

In [ ]:
df_tr = pd.read_csv(r'/content/drive/MyDrive/titanic/train.csv')
df_ts = pd.read_csv(r'/content/drive/MyDrive/titanic/test.csv')
temp = df_ts['PassengerId']
df_tr

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
df_tr.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
#PassengerId
list = [df_tr,df_ts]
for i in list:
    i.drop(['PassengerId'],axis=1,inplace=True)
df_tr.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
#Name
def name(x):
    x=re.findall("[a-zA-Z]+\.",x)
    return(str(x[0][:-1]))
    
keep = ['Mr','Mrs','Miss','Master']

for i in list:
    i['Name'] = i['Name'].apply(name)
    rem = [x for x in i['Name'].unique() if x not in keep]
    i['Name'].replace(rem,['Rare']*len(rem),inplace=True)
    le = LabelEncoder()
    i['Name'] = le.fit_transform(i['Name'])

In [ ]:
#Sex
for i in list:
    le = LabelEncoder()
    i['Sex'] = le.fit_transform(i['Sex'])

In [ ]:
#Age
gr = df_tr.groupby('Name')
df_tr['Age'] = gr['Age'].apply(lambda x: x.fillna(x.median()))
df_ts['Age'].fillna(df_ts['Age'].median(),inplace=True)

In [ ]:
#Ticket
for i in list:
    i.drop(['Ticket'],axis=1,inplace=True)
df_tr.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
#Cabin
for i in list:
    i.drop(['Cabin'],axis=1,inplace=True)
df_tr.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [ ]:
#Fare
df_ts['Fare'].fillna(df_ts['Fare'].median(),inplace=True)
for i in list:
  scaler = StandardScaler()
  i['Fare'] = scaler.fit_transform(i[['Fare']])

In [ ]:
#Embarked
for i in list:
    i['Embarked'].fillna(i['Embarked'].mode()[0],inplace=True)
    le = LabelEncoder()
    i['Embarked'] = le.fit_transform(i['Embarked'])

In [ ]:
df_tr.isna().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [ ]:
df_ts.isna().sum()

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [ ]:
X = df_tr.iloc[:,1:]
y = df_tr.iloc[:,0]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)

In [ ]:
def objective1(trial):
        kernel = trial.suggest_categorical('kernel',['linear','rbf','poly','sigmoid'])
        C = trial.suggest_loguniform('C', 1e-5, 1e2)
        degree = trial.suggest_int('degree', 1,4)
        clf = SVC(kernel=kernel, degree=degree,C=C)
        return sklearn.model_selection.cross_val_score(clf,X_train,y_train,n_jobs=-1, cv=3).mean()
    
def objective2(trial):    
        C = trial.suggest_loguniform('C', 1e-5, 1e2)
        clf = LogisticRegression(C=C)
        return sklearn.model_selection.cross_val_score(clf,X_train,y_train,n_jobs=-1, cv=3).mean()
        
def objective3(trial):
        n_estimators = trial.suggest_int('n_estimators',2,120)
        max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
        clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth)
        return sklearn.model_selection.cross_val_score(clf,X_train,y_train,n_jobs=-1, cv=3).mean()
    
def objective4(trial):
        n_estimators = trial.suggest_int('n_estimators',2,120)
        learning_rate = trial.suggest_discrete_uniform('learning_rate', 0.1, 1.0, 0.1)
        clf = GradientBoostingClassifier(n_estimators = n_estimators,learning_rate = learning_rate)
        return sklearn.model_selection.cross_val_score(clf,X_train,y_train,n_jobs=-1, cv=3).mean()
    
def objective5(trial):
        n_neighbors = trial.suggest_int('n_neighbors',3,30)
        weights = trial.suggest_categorical('weights',['uniform','distance'])
        clf = KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights)
        return sklearn.model_selection.cross_val_score(clf,X_train,y_train,n_jobs=-1, cv=3).mean()
      
def objective7(trial):
        layers = trial.suggest_int('layers',1,7)
        hidden_layer_sizes = []
        for i in range(layers):
          hidden_layer_sizes.append(trial.suggest_int('layer {} units: '.format(i+1),1,7))
        activation = trial.suggest_categorical('activation',['identity','logistic','tanh','relu'])
        clf = MLPClassifier(hidden_layer_sizes=tuple(hidden_layer_sizes), activation=activation)
        return sklearn.model_selection.cross_val_score(clf,X_train,y_train,n_jobs=-1, cv=3).mean()
    
def objective6(trial):
        learning_rate = trial.suggest_loguniform('learning_rate',1e-8, 1.0)
        max_depth = trial.suggest_int('max_depth', 1, 32)
        min_child_weight = trial.suggest_int('min_child_weight',1,5)
        clf = XGBClassifier(learning_rate=learning_rate,max_depth=max_depth,min_child_weight=min_child_weight)
        return sklearn.model_selection.cross_val_score(clf,X_train,y_train,n_jobs=-1, cv=3).mean()


In [ ]:
objectives = [objective1,objective2,objective3,objective4,objective5,objective6,objective7]
main_parameters = []

for i in objectives:
  study = optuna.create_study(direction='maximize')
  study.optimize(i,n_trials=30)
  trial = study.best_trial
  score = trial.value
  parameter = trial.params
  main_parameters.append((score,parameter))

[I 2021-05-31 10:11:40,319] A new study created in memory with name: no-name-cec0e756-cfee-4c9e-9664-3dbffabba6b7
[I 2021-05-31 10:11:40,385] Trial 0 finished with value: 0.7844099705086252 and parameters: {'kernel': 'rbf', 'C': 7.206143124097485, 'degree': 2}. Best is trial 0 with value: 0.7844099705086252.
[I 2021-05-31 10:11:40,619] Trial 1 finished with value: 0.7889481409660782 and parameters: {'kernel': 'linear', 'C': 0.3246759648459576, 'degree': 3}. Best is trial 1 with value: 0.7889481409660782.
[I 2021-05-31 10:11:40,656] Trial 2 finished with value: 0.6137707213940398 and parameters: {'kernel': 'linear', 'C': 1.0666878141996841e-05, 'degree': 3}. Best is trial 1 with value: 0.7889481409660782.
[I 2021-05-31 10:11:40,716] Trial 3 finished with value: 0.6137707213940398 and parameters: {'kernel': 'sigmoid', 'C': 0.01340805571013921, 'degree': 2}. Best is trial 1 with value: 0.7889481409660782.
[I 2021-05-31 10:11:40,778] Trial 4 finished with value: 0.6137707213940398 and para

In [ ]:
models = [SVC,LogisticRegression,RandomForestClassifier,GradientBoostingClassifier,KNeighborsClassifier,XGBClassifier]
print(main_parameters)

[(0.8248831791432689, {'kernel': 'rbf', 'C': 12.000150097151142, 'degree': 1}), (0.8084003285797007, {'C': 0.056210518961330586}), (0.829374217266594, {'n_estimators': 108, 'max_depth': 5.37976883025098}), (0.830842052815147, {'n_estimators': 20, 'learning_rate': 0.2}), (0.7634966805370392, {'n_neighbors': 3, 'weights': 'uniform'}), (0.8293472845042352, {'learning_rate': 0.03816428196814251, 'max_depth': 6, 'min_child_weight': 3}), (0.802434721717233, {'layers': 3, 'layer 1 units: ': 5, 'layer 2 units: ': 7, 'layer 3 units: ': 7, 'activation': 'identity'})]


In [ ]:
stack = []
strmodels = ['SVC','LogisticRegression','RandomForestClassifier','GradientBoostingClassifier','KNeighborsClassifier','XGBClassifier'] 
k = 0
for i in models:
  mod = i(**main_parameters[k][1])
  stack.append((strmodels[k],mod))
  print(stack[k])
  k+=1


layer = [x for x in main_parameters[-1][1].values()]
MLP = MLPClassifier(hidden_layer_sizes = tuple(layer[1:-1]), activation = str(layer[-1]))
NB = GaussianNB()
stack.append(('MLPClassifier',MLP))
stack.append(('GaussianNB',NB))
stack

('SVC', SVC(C=12.000150097151142, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=1, gamma='scale',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False))
('LogisticRegression', LogisticRegression(C=0.056210518961330586, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False))
('RandomForestClassifier', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5.37976883025098,
                       max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_sampl

[('SVC',
  SVC(C=12.000150097151142, break_ties=False, cache_size=200, class_weight=None,
      coef0=0.0, decision_function_shape='ovr', degree=1, gamma='scale',
      kernel='rbf', max_iter=-1, probability=False, random_state=None,
      shrinking=True, tol=0.001, verbose=False)),
 ('LogisticRegression',
  LogisticRegression(C=0.056210518961330586, class_weight=None, dual=False,
                     fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                     max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                     random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                     warm_start=False)),
 ('RandomForestClassifier',
  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='gini', max_depth=5.37976883025098,
                         max_features='auto', max_leaf_nodes=None,
                         max_samples=None, min_impurity_decrease=0.0,
                         m

In [ ]:
clf =  StackingClassifier(estimators=stack,final_estimator=LogisticRegression())
sklearn.model_selection.cross_val_score(clf,X_train,y_train,n_jobs=-1, cv=3).mean()

0.8293809504571836

In [ ]:
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: Conver

0.8251121076233184

In [ ]:
test = pd.DataFrame(clf.predict(df_ts))
test

,0
0,0
1,0
2,0
3,0
4,1
...,...
413,0
414,1
415,0
416,0


In [ ]:
test.set_index(temp,inplace=True)
test.rename({0:'Survived'},inplace=True,axis=1)
test

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [ ]:
test.to_csv(r'/content/drive/MyDrive/titanic/submission.csv')